In [ ]:
######################################################################################################
# Molina F., Chassin J., Camus H. (2024) Mistral AI for Developers Build Innovative Apps with Gen AI #
#                                                                                                    #
# This Notebook is a companion to the book. It provides the code examples and instructions needed to #
# replicate the concepts and techniques discussed in the book for integrating Mistral LLM into your  #
# projects and Google Cloud. Code for educational purpose.                                           #
######################################################################################################

In [ ]:
# install the required librairies
import sys
if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform > /dev/null
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown > /dev/null

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
# We start by generating synthetic marketing campaign data report that we store in a local sqlite db
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime

# Function to generate synthetic campaign data
def generate_campaign_data():
    data = {
        'campaign_id': np.random.randint(1000, 2000, size=10),
        'campaign_name': ['Campaign A', 'Summer Sale', 'Spring Promotion', 'Launch Event',
                          'Campaign B', 'Holiday Special', 'Campaign C', 'New Arrival',
                          'Summer Fest', 'Anniversary Sale'],
        'start_date': [datetime.now().strftime('%Y-%m-%d') for _ in range(10)],
        'end_date': [(datetime.now() + pd.offsets.DateOffset(days=np.random.randint(10,30))).strftime('%Y-%m-%d') for _ in range(10)],
        'impressions': np.random.randint(10000, 500000, size=10),
        'clicks': np.random.randint(30, 100, size=10),
        'conversions': np.random.randint(5, 50, size=10),
        'cost': np.random.randint(500, 5000, size=10)
    }
    return pd.DataFrame(data)

# Connect to the SQLite database (create if it doesn't exist)
conn = sqlite3.connect('marketing_data.db')

# Create a table if it doesn't exist
conn.execute('''
    CREATE TABLE IF NOT EXISTS campaigns (
        campaign_id INTEGER PRIMARY KEY,
        campaign_name TEXT,
        start_date DATE,
        end_date DATE,
        impressions INTEGER,
        clicks INTEGER,
        conversions INTEGER,
        cost INTEGER
    )
''')

# Generate and append synthetic data
df = generate_campaign_data()
df.to_sql('campaigns', conn, if_exists='append', index=False)

# Query the database and get data into a pandas DataFrame
query = "SELECT * FROM campaigns"
result_df = pd.read_sql_query(query, conn)

print(result_df)

# Close the connection
conn.close()

   campaign_id     campaign_name  start_date    end_date  impressions  clicks  \
0         1017        Campaign A  2024-01-22  2024-02-03        97233      87   
1         1150       Summer Fest  2024-01-22  2024-02-08       461506      47   
2         1238      Launch Event  2024-01-22  2024-02-04        91789      42   
3         1246       New Arrival  2024-01-22  2024-02-16       411676      93   
4         1297        Campaign B  2024-01-22  2024-02-10        78982      49   
5         1425  Anniversary Sale  2024-01-22  2024-02-09       308280      95   
6         1667  Spring Promotion  2024-01-22  2024-02-15       181626      52   
7         1888        Campaign C  2024-01-22  2024-02-13       486098      78   
8         1906       Summer Sale  2024-01-22  2024-02-09        11641      30   
9         1925   Holiday Special  2024-01-22  2024-02-11       498199      33   

   conversions  cost  
0           49  3785  
1           43  1628  
2           18  2604  
3           47  

In [ ]:
# (Here you can choose to use a Mistral Model deployed on a Vertex AI API endpoint, or refer to the previous notebook to see how to load Mistral's model locally in a notebook)

# Load an existing endpoint instance using the endpoint name
# You can set `ENDPOINT_ID = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789
from google.cloud import aiplatform

ENDPOINT_ID = "1234567890123456789"
PROJECT_ID = "... Your Project ID ..."
REGION = "us-central1"

AIP_ENDPOINT_ID = (
     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
)
endpoint = aiplatform.Endpoint(AIP_ENDPOINT_ID)

# Build the marketing report froma  table to a llm readable string of character
marketing_campaign_report = []
for i, row in result_df.iterrows():
  marketing_campaign_report.append("campaign id {0}, campaign name {1}, start_date {2}, end_date {3}, impressions {4}, clicks {5}, conversions {6}, cost $ {7}"
    .format(row["campaign_id"], row["campaign_name"], row["start_date"], row["end_date"], row["impressions"], row["clicks"], row["conversions"], row["cost"] ) )

# Print the report on llm readable format
print ("Synthetic Marketing Campaign Results:")
print(str.join("\n", marketing_campaign_report))

instances = [
    {
        "prompt": f"""<s>
                  [INST] Could you help me to analyse the marketing campaign performances? [/INST]
                    Of course! </s>
                  [INST] Which campaign has the best performances?
                  {marketing_campaign_report} [/INST]
                  """,
        "max_tokens": 1500,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Synthetic Marketing Campaign Results:
campaign id 1193, campaign name Launch Event, start_date 2024-01-21, end_date 2024-01-31, impressions 42048, clicks 87, conversions 397, cost $ 4810
campaign id 1612, campaign name Summer Sale, start_date 2024-01-21, end_date 2024-02-06, impressions 66673, clicks 90, conversions 202, cost $ 4725
campaign id 1666, campaign name Spring Promotion, start_date 2024-01-21, end_date 2024-02-08, impressions 410371, clicks 35, conversions 323, cost $ 3210
campaign id 1677, campaign name Anniversary Sale, start_date 2024-01-21, end_date 2024-02-08, impressions 444541, clicks 89, conversions 447, cost $ 1566
campaign id 1719, campaign name Holiday Special, start_date 2024-01-21, end_date 2024-02-05, impressions 405254, clicks 82, conversions 42, cost $ 4876
campaign id 1725, campaign name Campaign C, start_date 2024-01-21, end_date 2024-02-18, impressions 446680, clicks 68, conversions 371, cost $ 4887
campaign id 1764, campaign name New Arrival, start_date 2